# Code demonstration : Basis of Quantum Computing

### 1. Read-in implementations

Python code demonstrating the basis encoding for the integer 6.

In [1]:
import pennylane as qml

dev = qml.device("default.qubit", range(3))
@qml.qnode(dev)
def circuit(x):
    qml.BasisEmbedding(x, range(3))
    return qml.state()

# Call the function
circuit(6)

array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j])

Python code demonstrating amplitude encoding for a randomly generated complex vector.

In [2]:
import numpy as np
import pennylane as qml

# Number of qubits
n_qubits = 8

# Create a device with the specified number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(x):
    qml.AmplitudeEmbedding(features=x, wires=range(n_qubits), normalize=True, pad_with=0)
    return qml.state()

# Generate a random complex vector of size 2^n_qubits
x_real = np.random.normal(loc=0, scale=1, size=2**n_qubits)
x_imag = np.random.normal(loc=0, scale=1, size=2**n_qubits)
x = x_real + 1j * x_imag

# Execute the circuit to encode the vector as a quantum state
circuit(x)

array([-0.0034493 +0.04292555j,  0.10069397-0.03453532j,
        0.00251858-0.01652586j, -0.04962963-0.03967703j,
       -0.02342059+0.00158818j,  0.02161423-0.03263081j,
        0.05546708+0.01589991j, -0.01710984+0.03465088j,
        0.00619049-0.05730506j,  0.02852935-0.0360236j ,
        0.00947091+0.05771972j, -0.01295239-0.03264737j,
        0.00068972+0.03482173j, -0.00074203-0.0559428j ,
       -0.03814634-0.04456658j, -0.00267011+0.05494309j,
       -0.04063011-0.00076446j, -0.00368163-0.02577101j,
        0.11041359+0.05313858j,  0.01716034-0.00108382j,
        0.05707146+0.02917948j,  0.00904878-0.0294196j ,
        0.00734886-0.06484502j, -0.00219414+0.0372794j ,
        0.05925621-0.0467729j , -0.03379026-0.10502674j,
        0.0104578 -0.03931577j,  0.04734172-0.08597976j,
        0.03001382-0.03173529j,  0.01592617-0.02128152j,
       -0.0071303 +0.00029453j,  0.03005495+0.01988369j,
        0.03607238+0.01127004j, -0.00763043-0.02998118j,
       -0.06111551+0.05193207j,

Python code demonstrating angle encoding for a randomly generated real vector.

In [3]:
import numpy as np
import pennylane as qml 

# Number of qubits
n_qubits = 8

# Create a device with the specified number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(x):
    qml.AngleEmbedding(features=x, wires=range(n_qubits), rotation='X')
    return qml.state()

# Generate a random real vector of size n_qubits
x = np.random.uniform(0, np.pi, (n_qubits))

# Execute the circuit to encode the vector as a quantum state
circuit(x)

array([ 2.11857461e-03+0.00000000e+00j,  0.00000000e+00-1.50630120e-04j,
        0.00000000e+00-6.01041362e-03j, -4.27338891e-04+0.00000000e+00j,
        0.00000000e+00-2.16547255e-03j, -1.53964552e-04+0.00000000e+00j,
       -6.14346345e-03+0.00000000e+00j,  0.00000000e+00+4.36798701e-04j,
        0.00000000e+00-3.02360010e-03j, -2.14977204e-04+0.00000000e+00j,
       -8.57797838e-03+0.00000000e+00j,  0.00000000e+00+6.09892098e-04j,
       -3.09053218e-03+0.00000000e+00j,  0.00000000e+00+2.19736058e-04j,
        0.00000000e+00+8.76786524e-03j,  6.23393006e-04+0.00000000e+00j,
        0.00000000e+00-5.12915338e-05j, -3.64681511e-06+0.00000000e+00j,
       -1.45514504e-04+0.00000000e+00j,  0.00000000e+00+1.03460445e-05j,
       -5.24269515e-05+0.00000000e+00j,  0.00000000e+00+3.72754302e-06j,
        0.00000000e+00+1.48735693e-04j,  1.05750702e-05+0.00000000e+00j,
       -7.32025608e-05+0.00000000e+00j,  0.00000000e+00+5.20468360e-06j,
        0.00000000e+00+2.07676268e-04j,  1.47657302

### 2. Block encoding

Python code demonstrating the block encoding of a matrix via the linear combination *(LCU method)*. Note that the Pauli decomposition is time-consuming (for an N-qubit matrix it takes time $ \mathcal{O}(N 4^N)$) so not usable with large matrices.

In [5]:
import numpy as np
import pennylane as qml
import matplotlib.pyplot as plt

a = 0.36
b = 0.64

# Matrix to be decomposed
A = np.array(
    [[a, 0, 0, b],
     [0, -a, b, 0],
     [0, b, a, 0],
     [b, 0, 0, -a]]
)

# Decompose the matrix into sum of Pauli strings
LCU = qml.pauli_decompose(A)
LCU_coeffs, LCU_ops = LCU.terms()

# Normalized square roots of coefficients
alphas = (np.sqrt(LCU_coeffs)/ np.linalg.norm(np.sqrt(LCU_coeffs)))

dev = qml.device("default.qubit", wires=3)

# Unitaries
ops = LCU_ops

# Relabeling wires: 0 --> 1, and 1 --> 2
unitaries = [qml.map_wires(op, {0: 1, 1: 2}) for op in ops]

@qml.qnode(dev)
# Block encoding circuit
def lcu_circuit():
    # Preparation
    qml.StatePrep(alphas, wires=0)

    # Selection
    qml.Select(unitaries, control=0)

    # Dagger preparation
    qml.adjoint(qml.StatePrep(alphas, wires=0))
    return qml.state()

output_matrix = lcu_circuit()

print(np.real(np.round(output_matrix, 2)))

[ 0.36  0.    0.    0.64 -0.48  0.    0.    0.48]
